In [25]:
import pandas as pd
from transformers import XGLMTokenizer, XGLMForCausalLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig, XGLMTokenizerFast, XGLMConfig#, LlamaTokenizer, LlamaForCausalLM
import warnings
warnings.filterwarnings("ignore")

In [35]:
train_df =  pd.read_json("/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/train_ted_en-ja")
train_df

,talk_id,doc
0,129,"{'en': ['What I'm going to show you first, as ..."
1,769,{'en': ['I'd like to share with you a discover...
2,1666,"{'en': ['When I was 11, I remember waking up ..."
3,1160,{'en': ['This is a representation of your brai...
4,1161,"{'en': ['I'm Jessi, and this is my suitcase.',..."
...,...,...
60,1144,"{'en': ['My name is Amit.', 'And 18 months ago..."
61,1659,"{'en': ['I have never, ever forgotten the word..."
62,1404,"{'en': ['This may sound strange, but I'm a bi..."
63,1534,"{'en': ['So, well, I do applied math, and thi..."


In [36]:
test_df = pd.read_json("/home/sumire/thesis/LLM_Contextual_Prompt_MT/data/iwslt_hf/test_ted_en-ja")
test_df

,talk_id,doc
0,1922,"{'en': ['Intelligence -- what is it?', 'If we ..."
1,2183,{'en': ['Today I'm going to speak to you abou...
2,1932,{'en': ['I'm going to talk to you tonight abo...
3,1939,"{'en': ['I had brain surgery 18 years ago, an..."
4,1954,{'en': ['In many patriarchal societies and tri...
5,1443,{'en': ['I'd like to invite you to close your ...
6,1829,"{'en': ['So I'm going to talk about trust, an..."
7,1961,{'en': ['The world makes you something that yo...
8,1835,"{'en': ['So, we used to solve big problems.', ..."
9,2102,"{'en': ['This is my niece, Stella.', 'She's ju..."


In [33]:
tokenizer = XGLMTokenizer.from_pretrained("facebook/xglm-7.5B")
train_num_docs = len(train_df)
test_num_docs = len(test_df)

train_num_sents = 0
train_all_num_tokens = 0
train_num_tokens_list = []
for doc in train_df["doc"]:
    num_sent = len(doc["en"])
    train_num_sents += num_sent
    for sent in doc["en"]:
        num_tokens = len(tokenizer(sent).input_ids)
        train_all_num_tokens += num_tokens
        train_num_tokens_list.append(num_tokens)

train_avg_num_tokens = train_all_num_tokens/train_num_sents
train_max_tokens_per_sent = max(train_num_tokens_list)


test_num_sents = 0    
test_all_num_tokens = 0
test_num_tokens_list = []
for doc in test_df["doc"]:
    sent = []
    num_sent = len(doc["en"])
    test_num_sents += num_sent
    for sent in doc["en"]:
        num_tokens = len(tokenizer(sent).input_ids)
        test_all_num_tokens += num_tokens
        test_num_tokens_list.append(num_tokens)
test_avg_num_tokens = train_all_num_tokens/test_num_sents
test_max_tokens_per_sent = max(test_num_tokens_list)

In [34]:
pd.DataFrame(
    {
        "num_doc":[train_num_docs, test_num_docs],
        "num_sent": [train_num_sents, test_num_sents],
        "max_tokens_per_sent":[train_max_tokens_per_sent, test_max_tokens_per_sent],
        "avg_tokens_per_sent":[train_avg_num_tokens, test_avg_num_tokens],
    }, 
    index=["train", "test"]
)

,num_doc,num_sent,max_tokens_per_sent,avg_tokens_per_sent
train,65,6471,114,22.779632
test,27,2479,107,59.462283
